In [1]:
# Start small and save your time.

In [2]:
# import necessary packages

import pandas as pd # for working with tables
pd.options.mode.chained_assignment = None  # default = 'warn'

import os # for finding file directory with os.getcwd()

# # for calculations
import numpy as np 
import math

import matplotlib.pyplot as plt # for plotting

# # for plotting histogram with percentages
from matplotlib.ticker import PercentFormatter
import matplotlib.ticker as mtick

from PIL import Image # for converting plots to black-and-white

In [3]:
from ipynb.fs.full.MyFunctions_00 import *

In [ ]:
voids_6arc = read_data('IPAC_Results/New/Voids/new_voids_AWSource')
voids_6arc = voids_6arc.rename(columns = {'nsaid_01' : 'object_tag', 
                                          'w1mpro' : 'w1mpro_AWS',
                                          'w2mpro' : 'w2mpro_AWS'})

print(len(voids_6arc))
voids_filtered = filter_colors(voids_6arc)
print(len(voids_filtered))

voids_filtered_cols = voids_filtered[["object_tag", "w1mpro_AWS", "w2mpro_AWS",]]
voids_filtered_cols.head()


In [ ]:
walls_6arc = read_data('IPAC_Results/New/Walls/new_walls_AWSource')
walls_6arc = walls_6arc.rename(columns = {'nsaid_01' : 'object_tag', 
                                          'w1mpro' : 'w1mpro_AWS',
                                          'w2mpro' : 'w2mpro_AWS'})

print(len(walls_6arc))
walls_filtered = filter_colors(walls_6arc)
print(len(walls_filtered))

walls_filtered_cols = walls_filtered[["object_tag", "w1mpro_AWS", "w2mpro_AWS",]]
walls_filtered_cols.head()


## Compile for VOIDS - Done as of ______

In [ ]:
# import data from NEOWISE and ALLWISE MEP (multi-epoch photometry)

# since the NEOWISE data files are so large, import each piece of the data for each group and then bind together
voids_NEO_1 = read_data('IPAC_Results/AWS/Voids/ONE_AWS_voids_NEOWISE')
voids_NEO_12 = read_data('IPAC_Results/AWS/Voids/ONE2_AWS_voids_NEOWISE')
voids_NEO_2 = read_data('IPAC_Results/AWS/Voids/TWO_AWS_voids_NEOWISE')
voids_NEO_22 = read_data('IPAC_Results/AWS/Voids/TWO2_AWS_voids_NEOWISE')
voids_NEO_3 = read_data('IPAC_Results/AWS/Voids/THREE_AWS_voids_NEOWISE')

voids_NEO_full = pd.concat([voids_NEO_1, voids_NEO_12, 
                            voids_NEO_2, voids_NEO_22,
                            voids_NEO_3], 
                           ignore_index = True, axis = 0)

# voids_NEO_full = voids_NEO_full.head()

voids_ME_1 = read_data('IPAC_Results/AWS/Voids/ONE_AWS_voids_MEP')
voids_ME_2 = read_data('IPAC_Results/AWS/Voids/TWO_AWS_voids_MEP')
voids_ME_3 = read_data('IPAC_Results/AWS/Voids/THREE_AWS_voids_MEP')

voids_ME_full = pd.concat([voids_ME_1, voids_ME_2, voids_ME_3], ignore_index = True, axis = 0)

In [ ]:
voids_NEO_1.head()

In [ ]:
voids_ME_full.head()

In [ ]:
num_voids_NEO = np.unique(voids_NEO_full['nsaid_01'])
print(len(num_voids_NEO))

num_voids_ME = np.unique(voids_ME_full['nsaid_01'])
print(len(num_voids_ME))

num_voids_both = np.concatenate((num_voids_NEO, num_voids_ME), axis=None)
print(len(num_voids_both))

num_voids_unique = np.unique(num_voids_both)
print(len(num_voids_unique))
print('')
print(len(num_voids_unique[~np.isnan(num_voids_unique)]))

In [ ]:
# implement the above functions for the voids dataset
voids_full = combine_new_colors(voids_NEO_full, voids_ME_full, voids_filtered_cols)
tags_unique_voids = np.unique(voids_full['object_tag'])

voids_full.head()

In [ ]:
len(tags_unique_voids)

In [ ]:
new_voids = voids_full

new_voids_list = read_data_db('new_voids_filtDL_04-10-23')

new_voids_list = new_voids_list.rename(columns = {'NSAID' : 'nsaid_01'})

new_voids_merged = new_voids.merge(new_voids_list, how = 'left', on = ['nsaid_01'])

new_voids_filt = new_voids_merged.dropna(subset = ['RA'])

voids_full = new_voids_filt

print(len(np.unique(voids_full['object_tag'])))

In [ ]:
# redo tag list after filtering
tags_unique_voids = np.unique(voids_full['object_tag'])
print(len(tags_unique_voids))

# problem = nan value!
print(tags_unique_voids.min())
print(tags_unique_voids.max())

In [ ]:
# remove nan value
tags_unique_voids = tags_unique_voids[np.logical_not(np.isnan(tags_unique_voids))]
print(len(tags_unique_voids))

In [ ]:
# make an empty mean table
# using a for loop, add the rows from each unique object's mean table into the ultimate mean table
# the ultimate_mean table will contain the information needed to plot the light curve for every void object
ultimate_mean_voids = pd.DataFrame(columns = ['object_tag', 'mjd_binned',
                                              'mean_W1', 'mean_W2', 'mean_color',
                                              'std_W1', 'std_W2', 'std_color'])

outliers_removed_voids = pd.DataFrame(columns = ['nsaid_01',
                                                     'w1mpro',
                                                     'w2mpro',
                                                     'mjd',
                                                     'object_tag',
                                                     'origin',
                                                     'w1mpro_AWS',
                                                     'w2mpro_AWS',
                                                     'W2_diff',
                                                     'mjd_zero',
                                                     'W1-W2',
                                                     'bin_num',
                                                     'mjd_binned',
                                                     'med_dev_W1',
                                                     'med_dev_W2',
                                                     'dev_score_W1',
                                                     'dev_score_W2'])

count = 0

for t in tags_unique_voids:
    voids_full_tag = expand_cols(voids_full, t)
#     voids_mean = save_binned(voids_full_tag, t)

    voids_tuple = save_binned_outliers(voids_full_tag, t)
    voids_mean = voids_tuple[0]
    voids_outliers_removed = voids_tuple[1]
    
    ultimate_mean_voids = pd.concat([ultimate_mean_voids, voids_mean], ignore_index = True)
    outliers_removed_voids = pd.concat([outliers_removed_voids, voids_outliers_removed], ignore_index = True)

    count += 1
    
    # increment the counter to help us track the progress of this loop (takes a while to run)
    # print the counter after every 5000 objects have been processed
    if(count%5000 == 0):
        print(str(count) + ' voids')
        
# 65000 voids

In [ ]:
# voids_full[voids_full['object_tag'].isna()]

In [ ]:
print(len(ultimate_mean_voids))
ultimate_mean_voids.head()

ultimate_mean_voids.to_csv('Files/ultimate_mean_VOIDS_' + 'new_' + timestamp() + '.csv', index = False)
outliers_removed_voids.to_csv('Files/bin_outliers_removed_VOIDS_' + timestamp() + '.csv', index = False)
# voids_full.to_csv('Files/voids_full_' + 'new_' + timestamp() + '.csv', index = False)

## Compile for WALLS: Done as of _______

In [ ]:
walls_NEO_1 = read_data('IPAC_Results/AWS/Walls/ONE_AWS_walls_NEOWISE')
walls_NEO_2 = read_data('IPAC_Results/AWS/Walls/TWO_AWS_walls_NEOWISE')
walls_NEO_3 = read_data('IPAC_Results/AWS/Walls/THREE_AWS_walls_NEOWISE')
walls_NEO_4 = read_data('IPAC_Results/AWS/Walls/FOUR_AWS_walls_NEOWISE')
walls_NEO_5 = read_data('IPAC_Results/AWS/Walls/FIVE_AWS_walls_NEOWISE')
walls_NEO_6 = read_data('IPAC_Results/AWS/Walls/SIX_AWS_walls_NEOWISE')
walls_NEO_7 = read_data('IPAC_Results/AWS/Walls/SEVEN_AWS_walls_NEOWISE')

walls_NEO_12 = read_data('IPAC_Results/AWS/Walls/ONE2_AWS_walls_NEOWISE')
walls_NEO_22 = read_data('IPAC_Results/AWS/Walls/TWO2_AWS_walls_NEOWISE')
walls_NEO_32 = read_data('IPAC_Results/AWS/Walls/THREE2_AWS_walls_NEOWISE')
walls_NEO_42 = read_data('IPAC_Results/AWS/Walls/FOUR2_AWS_walls_NEOWISE')
walls_NEO_52 = read_data('IPAC_Results/AWS/Walls/FIVE2_AWS_walls_NEOWISE')
walls_NEO_62 = read_data('IPAC_Results/AWS/Walls/SIX2_AWS_walls_NEOWISE')
walls_NEO_72 = read_data('IPAC_Results/AWS/Walls/SEVEN2_AWS_walls_NEOWISE')

walls_NEO_full = pd.concat([walls_NEO_1, walls_NEO_12, 
                            walls_NEO_2, walls_NEO_22,  
                            walls_NEO_3, walls_NEO_32,  
                            walls_NEO_4, walls_NEO_42,  
                            walls_NEO_5, walls_NEO_52, 
                            walls_NEO_6, walls_NEO_62, 
                            walls_NEO_7, walls_NEO_72],
                           ignore_index = True, axis = 0)

walls_ME_1 = read_data('IPAC_Results/AWS/Walls/ONE_AWS_walls_MEP')
walls_ME_2 = read_data('IPAC_Results/AWS/Walls/TWO_AWS_walls_MEP')
walls_ME_3 = read_data('IPAC_Results/AWS/Walls/THREE_AWS_walls_MEP')
walls_ME_4 = read_data('IPAC_Results/AWS/Walls/FOUR_AWS_walls_MEP')
walls_ME_5 = read_data('IPAC_Results/AWS/Walls/FIVE_AWS_walls_MEP')
walls_ME_6 = read_data('IPAC_Results/AWS/Walls/SIX_AWS_walls_MEP')
walls_ME_7 = read_data('IPAC_Results/AWS/Walls/SEVEN_AWS_walls_MEP')

walls_ME_full = pd.concat([walls_ME_1, 
                            walls_ME_2, 
                            walls_ME_3, 
                            walls_ME_4, 
                            walls_ME_5,
                            walls_ME_6,
                            walls_ME_7],
                           ignore_index = True, axis = 0)

In [ ]:
walls_NEO_1.head()

In [ ]:
walls_ME_full.head()

In [ ]:
print(round(walls_NEO_full[(walls_NEO_full['nsaid_01'] == 552295)]['mjd'].min(), 3))

walls_NEO_full[(walls_NEO_full['nsaid_01'] == 552295)]

In [ ]:
print(round(walls_ME_full[(walls_ME_full['nsaid_01'] == 552295)]['mjd'].min(), 3))

walls_ME_full[(walls_ME_full['nsaid_01'] == 552295)]

In [ ]:
num_walls_NEO = np.unique(walls_NEO_full['nsaid_01'])
print(len(num_walls_NEO))

num_walls_ME = np.unique(walls_ME_full['nsaid_01'])
print(len(num_walls_ME))

num_walls_both = np.concatenate((num_walls_NEO, num_walls_ME), axis=None)
print(len(num_walls_both))

num_walls_unique = np.unique(num_walls_both)
print(len(num_walls_unique))
print('')
print(len(num_walls_unique[~np.isnan(num_walls_unique)]))
print(num_walls_unique.max())

In [ ]:
walls_full = combine_new_colors(walls_NEO_full, walls_ME_full, walls_filtered_cols)
tags_unique_walls = np.unique(walls_full['object_tag'])

In [ ]:
len(tags_unique_walls)

In [ ]:
new_walls = walls_full

new_walls_list = read_data_db('new_walls_filtDL_04-10-23')

new_walls_list = new_walls_list.rename(columns = {'NSAID' : 'nsaid_01'})

new_walls_merged = new_walls.merge(new_walls_list, how = 'left', on = ['nsaid_01'])

new_walls_filt = new_walls_merged.dropna(subset = ['RA'])

walls_full = new_walls_filt

print(len(np.unique(walls_full['object_tag'])))

In [ ]:
# redo tag list after filtering
tags_unique_walls = np.unique(walls_full['object_tag'])
print(len(tags_unique_walls))

# problem = nan value!
print(tags_unique_walls.min())
print(tags_unique_walls.max())

In [ ]:
tags_unique_walls = tags_unique_walls[np.logical_not(np.isnan(tags_unique_walls))]
print(len(tags_unique_walls))

In [ ]:
# repeat for walls
ultimate_mean_walls = pd.DataFrame(columns = ['object_tag', 'mjd_binned',
                                              'mean_W1', 'mean_W2', 'mean_color',
                                              'std_W1', 'std_W2', 'std_color'])

outliers_removed_walls = pd.DataFrame(columns = ['nsaid_01',
                                                     'w1mpro',
                                                     'w2mpro',
                                                     'mjd',
                                                     'object_tag',
                                                     'origin',
                                                     'w1mpro_AWS',
                                                     'w2mpro_AWS',
                                                     'W2_diff',
                                                     'mjd_zero',
                                                     'W1-W2',
                                                     'bin_num',
                                                     'mjd_binned',
                                                     'med_dev_W1',
                                                     'med_dev_W2',
                                                     'dev_score_W1',
                                                     'dev_score_W2'])

count = 0

for t in tags_unique_walls:
    walls_full_tag = expand_cols(walls_full, t)
#     walls_mean = save_binned(walls_full_tag, t)

    walls_tuple = save_binned_outliers(walls_full_tag, t)
    walls_mean = walls_tuple[0]
    walls_outliers_removed = walls_tuple[1]
    
    ultimate_mean_walls = pd.concat([ultimate_mean_walls, walls_mean], ignore_index = True)
    outliers_removed_walls = pd.concat([outliers_removed_walls, walls_outliers_removed], ignore_index = True)

    count += 1
    
    # increment the counter to help us track the progress of this loop (takes a while to run)
    # print the counter after every 5000 objects have been processed
    if(count%5000 == 0):
        print(str(count) + ' walls')
        
# 270,000 walls

In [ ]:
print(len(ultimate_mean_walls))
ultimate_mean_walls.head()

ultimate_mean_walls.to_csv('Files/ultimate_mean_walls_' + 'new_' + timestamp() + '.csv', index = False)
outliers_removed_walls.to_csv('Files/bin_outliers_removed_walls_' + timestamp() + '.csv', index = False)
# walls_full.to_csv('Files/walls_full_' + 'new_' + timestamp() + '.csv', index = False)